In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

In [ ]:
#only required to preprocess once
ini_path="train_dataset"
for (dirpath,dirnames,filenames) in os.walk(ini_path):
    for dirname in dirnames:
            for(direcpath,direcnames,files) in os.walk(ini_path+"\\"+dirname):
                i=0
                for file in files:
                        
                    actual_path=ini_path+"\\\\"+dirname+"\\\\"+file
        
                    frame = cv2.imread(actual_path)
                    frame = cv2.resize(frame,(64,64))
                    
                    converted = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) # Convert from RGB to HSV

                    lowerBoundary = np.array([0,40,30],dtype="uint8")
                    upperBoundary = np.array([43,255,254],dtype="uint8")

                    skinMask = cv2.inRange(converted, lowerBoundary, upperBoundary)

                    skinMask = cv2.medianBlur(skinMask, 5)
                    skin = cv2.bitwise_and(frame, frame, mask = skinMask)
                    
                    bw_image = cv2.cvtColor(skin, cv2.COLOR_HSV2BGR)  
                    bw_image = cv2.cvtColor(skin, cv2.COLOR_BGR2GRAY)  
                    bw_image = cv2.GaussianBlur(bw_image,(5,5),0)  
                    
                    if(i%5==0):
                        directory="test_processed"
                    else:
                        directory="train_processed"
                    
                    path = directory+"\\\\"+dirname+"\\\\"
                    curr_path=path+file
                    cv2.imwrite(curr_path, bw_image)
                    i=i+1

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
Imagesize=64
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(Imagesize,Imagesize,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.25))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(25, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

In [ ]:
model.summary()

In [ ]:
data_train = ImageDataGenerator(rescale=1.0/255.0,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,
                                shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
data_valid = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
batch = 96

In [ ]:
train_gen = data_train.flow_from_directory('train_processed',target_size=(Imagesize,Imagesize),batch_size=batch, 
                                           class_mode='categorical')

test_gen = data_valid.flow_from_directory('test_processed', target_size=(Imagesize,Imagesize), batch_size=batch, 
                                           class_mode='categorical')

In [ ]:
model.fit_generator(train_gen, epochs=3, steps_per_epoch=50, validation_data = test_gen, 
                    validation_steps=4, callbacks=[reduce_lr, early_stop])

In [ ]:
model.save('islcnnmodel11.h5')

In [ ]:
imgs,labels=next(test_gen)
scores = model.evaluate(imgs,labels,verbose=0)
print("Accuracy: ",scores[1])

In [ ]:
Y_pred = model.predict_generator(test_gen,len(test_gen.filenames)//batch+1)
y_pred = np.argmax(Y_pred, axis=1)
import seaborn as sns
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_gen.classes, y_pred)
sns.set(font_scale=1)
plt.figure(figsize = (12,8))
sns.heatmap(cm, annot=True,fmt='g',cmap='PuBu')

In [ ]:
def prepare(filepath):
    img_array = cv2.imread(filepath)
    return img_array.reshape(1, Imagesize, Imagesize, 3)

for i in range(25):
    if(i>=9):
        prediction = model.predict_classes([prepare("test_processed\\"+chr(i+65+1)+"\\0.jpg")])
        if chr(i+65+1)==chr(prediction+65+1):
            print(chr(prediction+65+1))
    else:
        prediction = model.predict_classes([prepare("test_processed\\"+chr(i+65)+"\\0.jpg")])
        if chr(i+65)==chr(prediction+65):
            print(chr(prediction+65))